<a href="https://colab.research.google.com/github/Dipti009/Dipti009/blob/main/Copy_of_Lighting_thunder_animetion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# ⚡️ Realistic Lightning Simulator – One-Click Colab Cell
# ============================================================
# 1. Installs everything (40 s on first run)
# 2. Generates branching lightning with glow + thunder
# 3. Streams live preview + exports MP4/GIF
# 4. Serves a public Gradio URL instantly
# ------------------------------------------------------------

# ---------- 0. one-time setup ----------
import os, sys, math, random, tempfile, subprocess, requests, base64, json
if not os.path.exists("/tmp/.lightning_ready"):
    !pip install -q gradio moviepy opencv-python-headless numpy
    open("/tmp/.lightning_ready", "w").write("done")

# ---------- 1. imports ----------
import cv2, numpy as np
from moviepy.editor import ImageSequenceClip, AudioFileClip
import gradio as gr

# ---------- 2. fast, deterministic lightning ----------
def lerp(a, b, t): return a + t * (b - a)
def dist(p1, p2):  return math.hypot(p1[0]-p2[0], p1[1]-p2[1])
def midpoint(p1, p2): return ((p1[0]+p2[0])//2, (p1[1]+p2[1])//2)

def build_lightning_path(start, end, rough=0.2, branch_prob=0.35):
    branches = []
    def subdivide(p1, p2, depth=0):
        if dist(p1, p2) < 5 or depth > 8: return [p2]
        m = midpoint(p1, p2)
        m = (m[0] + int(random.uniform(-1, 1) * rough * dist(p1, p2)), m[1])
        path = subdivide(p1, m, depth+1) + subdivide(m, p2, depth+1)
        if random.random() < branch_prob and depth < 4:
            branch_len = random.randint(20, 80)
            angle = random.uniform(-math.pi/2, math.pi/2)
            branch_end = (m[0] + int(branch_len*math.cos(angle)),
                          m[1] + int(branch_len*math.sin(angle)))
            branches.append(subdivide(m, branch_end, depth+1))
        return path
    main = subdivide(start, end)
    return [main] + branches

def draw_paths(canvas, paths, color, thickness):
    for path in paths:
        for i in range(len(path)-1):
            cv2.line(canvas, path[i], path[i+1], color, thickness)

def add_glow(canvas, paths, color, glow_radius=15):
    glow = canvas.copy()
    draw_paths(glow, paths, color, glow_radius)
    glow = cv2.GaussianBlur(glow, (0,0), glow_radius)
    return cv2.addWeighted(canvas, 1, glow, 0.6, 0)

def generate_lightning_video(w, h, dur, fps, color, branch, rough, style, gif):
    frames = dur * fps
    out = []
    for f in range(frames):
        alpha = max(0, 1 - (f/(fps*0.4))) if style == "normal" else (1 if f%6<3 else 0)
        img = np.zeros((h, w, 3), dtype=np.uint8)
        if alpha > 0:
            start = (w//2, 0)
            end   = (random.randint(w//3, 2*w//3), h-20)
            paths = build_lightning_path(start, end, rough, branch)
            c = tuple(int(c*alpha) for c in color)
            draw_paths(img, paths, c, 2)
            img = add_glow(img, paths, c, 15)
        out.append(img)

    mp4_path = tempfile.mktemp(suffix=".mp4")
    clip = ImageSequenceClip(out, fps=fps)
    # optional thunder
    try:
        snd = "/tmp/thunder.mp3"
        if not os.path.exists(snd):
            url = "https://cdn.freesound.org/previews/377/377172_5720237-lq.mp3"
            open(snd, "wb").write(requests.get(url, timeout=10).content)
        clip = clip.set_audio(AudioFileClip(snd).subclip(0, clip.duration).audio_fadeout(0.5))
    except Exception as e:
        print("Thunder audio skipped:", e)
    clip.write_videofile(mp4_path, verbose=False, logger=None)

    gif_path = None
    if gif:
        gif_path = mp4_path.replace(".mp4", ".gif")
        subprocess.run(["ffmpeg", "-y", "-i", mp4_path,
                        "-vf", "fps=15,scale=320:-1:flags=lanczos", gif_path],
                       stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return mp4_path, gif_path

# ---------- 3. Gradio UI ----------
presets = {
    "Classic Storm": {"color": (220, 235, 255), "branch": 0.35, "rough": 0.2},
    "Purple Blaze":  {"color": (200, 0, 255),   "branch": 0.45, "rough": 0.25},
    "Hellfire":      {"color": (255, 90, 0),    "branch": 0.5,  "rough": 0.3},
    "Ice Crack":     {"color": (0, 220, 255),   "branch": 0.25, "rough": 0.15},
    "Strobe":        {"color": (255, 255, 255), "branch": 0.2,  "rough": 0.05},
}

css = """
body {background:#111}
.gr-button {background:#ffcc00;color:#000;font-weight:bold}
"""

with gr.Blocks(css=css, title="⚡ Lightning Studio") as demo:
    gr.Markdown("# ⚡ Realistic Lightning Generator")
    with gr.Row():
        with gr.Column(scale=2):
            preset   = gr.Dropdown(list(presets.keys()), value="Classic Storm", label="Preset")
            w        = gr.Slider(320, 1280, 640, step=10, label="Width")
            h        = gr.Slider(240, 720, 480, step=10, label="Height")
            dur      = gr.Slider(0.5, 5, 2, step=0.1, label="Duration (s)")
            fps      = gr.Slider(10, 60, 30, step=1, label="FPS")
            color    = gr.ColorPicker(value="#DCEBFF", label="Color")
            branch   = gr.Slider(0, 1, 0.35, step=0.05, label="Branch Probability")
            rough    = gr.Slider(0, 1, 0.2, step=0.05, label="Roughness")
            style    = gr.Radio(["normal", "strobe"], value="normal", label="Style")
            gif      = gr.Checkbox(value=False, label="Also create GIF")
            go_btn   = gr.Button("Generate ⚡", variant="primary")
        with gr.Column(scale=3):
            video    = gr.Video(label="MP4")
            gif_out  = gr.Image(label="GIF", visible=False)

    def run(p, w, h, dur, fps, color, branch, rough, style, gif):
        rgb = tuple(int(color.lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
        if p in presets:
            pr = presets[p]
            branch, rough, rgb = pr["branch"], pr["rough"], pr["color"]
        mp4, gif_file = generate_lightning_video(w, h, dur, fps, rgb, branch, rough, style, gif)
        return mp4, gif_file if gif else None

    go_btn.click(run,
                 inputs=[preset, w, h, dur, fps, color, branch, rough, style, gif],
                 outputs=[video, gif_out])

    gr.Examples([
        ["Classic Storm", 640, 480, 2, 30, "#DCEBFF", 0.35, 0.2, "normal", False],
        ["Purple Blaze", 512, 512, 1.5, 30, "#C800FF", 0.45, 0.25, "normal", True],
        ["Hellfire", 720, 720, 1, 24, "#FF5A00", 0.5, 0.3, "strobe", False],
    ], fn=run, inputs=[preset, w, h, dur, fps, color, branch, rough, style, gif],
       outputs=[video, gif_out], cache_examples=False)

demo.queue().launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8d9df33f8cdb8dc54b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Thunder audio skipped: MoviePy error: failed to read the duration of file /tmp/thunder.mp3.
Here are the file infos returned by ffmpeg:

ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-lib

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2250, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1757, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Thunder audio skipped: MoviePy error: failed to read the duration of file /tmp/thunder.mp3.
Here are the file infos returned by ffmpeg:

ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-lib